In [2]:
# A ScratchBook Created by me to explore TOP Data
# Step 1: Get all data unzipped under ~/data
import os
import zipfile

def unzip_files(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".zip"):
                file_path = os.path.join(root, file)
                try:
                    if file.startswith("EAGLE"):
                        with zipfile.ZipFile(file_path, 'r') as zip_ref:
                            zip_ref.extractall(root)
                        print(f"Extracted {file_path}")
                    elif file.startswith("ODIN"):
                        with zipfile.ZipFile(file_path, 'r') as zip_ref:
                            zip_ref.extractall(root)
                        print(f"Extracted {file_path}")
                    else:
                        print("This should not execute")
                except Exception as e:
                    print(f"Error extracting {file_path}: {e}")

if __name__ == "__main__":
    target_directory = "./data"
    if os.path.isdir(target_directory):
        unzip_files(target_directory)
        print(f"Files in {target_directory} have been unzipped")
    else:
        print(f"The specified directory does not exist")


Extracted ./data/EAGLE-I-20230928T215757Z-001.zip
Extracted ./data/EAGLE-I-20230928T215757Z-002.zip
Extracted ./data/EAGLE-I-20230928T215757Z-003.zip
Extracted ./data/EAGLE-I-20230928T215757Z-004.zip
Extracted ./data/ODIN-20230928T215912Z-001.zip
Files in ./data have been unzipped


In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
import matplotlib.pyplot as plt
import json

In [19]:
# Load all ODIN data values
ODIN_county = pd.read_csv("./data/ODIN/odin_county_outage_data.csv")
ODIN_polygon = pd.read_csv("./data/ODIN/odin_polygon_outage_data.csv")
ODIN_zip = pd.read_csv("./data/ODIN/odin_zip_outage_data.csv")

In [3]:
ODIN_county.columns, ODIN_polygon.columns, ODIN_zip.columns

(Index(['reported_time', 'fips_county', 'meters_out', 'us_state'], dtype='object'),
 Index(['reported_time', 'geojson', 'meters_out', 'us_state'], dtype='object'),
 Index(['reported_time', 'zip_code', 'meters_out', 'us_state'], dtype='object'))

In [4]:
ODIN_polygon

,reported_time,geojson,meters_out,us_state
0,2022-07-06 05:53:16.638+00,"{""type"":""Polygon"",""coordinates"":[[[-122.167300...",2,Washington
1,2022-09-26 09:50:13.384+00,"{""type"":""Polygon"",""coordinates"":[[[-121.336660...",5,Washington
2,2022-09-26 09:50:13.384+00,"{""type"":""Polygon"",""coordinates"":[[[-122.530725...",2,Washington
3,2022-09-26 09:50:13.384+00,"{""type"":""Polygon"",""coordinates"":[[[-122.634957...",5,Washington
4,2022-09-26 09:55:06.186+00,"{""type"":""Polygon"",""coordinates"":[[[-121.336660...",5,Washington
...,...,...,...,...
659885,2023-09-05 17:34:12.073+00,"{""type"":""Polygon"",""coordinates"":[[[-122.544762...",1,Washington
659886,2023-09-05 17:34:12.073+00,"{""type"":""Polygon"",""coordinates"":[[[-122.618902...",63,Washington
659887,2023-09-05 17:34:12.073+00,"{""type"":""Polygon"",""coordinates"":[[[-122.829829...",16,Washington
659888,2023-09-05 17:34:12.073+00,"{""type"":""Polygon"",""coordinates"":[[[-122.885263...",4,Washington


In [20]:
# Convert the reported_time to standard datetime
ODIN_polygon['reported_time'] = pd.to_datetime(ODIN_polygon['reported_time'], format="ISO8601")

# Parse and modify the 'type' field in geojson
for i in range(len(ODIN_polygon['geojson'])):
    geojson_dict = json.loads(ODIN_polygon.at[i, 'geojson'])
    geojson_dict['type'] = "Polygons"
    ODIN_polygon.at[i, 'geojson'] = json.dumps(geojson_dict)

ODIN_polygon['geojson'] = ODIN_polygon['geojson']#.apply(json.loads)

# Create a GeoDataFrame for mapping GeoJSON
# outages_gdf = gpd.GeoDataFrame(ODIN_polygon, geometry=gpd.GeoSeries.from_wkt(ODIN_polygon['geojson']))
outages_gdf = gpd.GeoDataFrame.from_features(ODIN_polygon.to_dict('records'))

KeyError: 'geometry'

In [14]:
ODIN_polygon

,reported_time,geojson,meters_out,us_state
0,2022-07-06 05:53:16.638000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-122.16...",2,Washington
1,2022-09-26 09:50:13.384000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-121.33...",5,Washington
2,2022-09-26 09:50:13.384000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-122.53...",2,Washington
3,2022-09-26 09:50:13.384000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-122.63...",5,Washington
4,2022-09-26 09:55:06.186000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-121.33...",5,Washington
...,...,...,...,...
659885,2023-09-05 17:34:12.073000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-122.54...",1,Washington
659886,2023-09-05 17:34:12.073000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-122.61...",63,Washington
659887,2023-09-05 17:34:12.073000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-122.82...",16,Washington
659888,2023-09-05 17:34:12.073000+00:00,"{""type"": ""Polygons"", ""coordinates"": [[[-122.88...",4,Washington


## End ODIN

In [5]:
EAGLE_coverageHistory = pd.read_csv('./data/EAGLE-I/coverage_history.csv')
EAGLE_outages2020 = pd.read_csv('./data/EAGLE-I/eaglei_outages_2020.csv')

In [6]:
cdc_socialVulnerability2020 = pd.read_csv('./data/5-CDC-Social-Vulnerability/svi_interactive_map.csv')

In [7]:
EAGLE_outages2020.columns

Index(['fips_code', 'county', 'state', 'customers_out', 'run_start_time'], dtype='object')

In [8]:
cdc_socialVulnerability2020.columns

Index(['GEO LEVEL', 'GEO COMPARISON', 'YEAR', 'GEO UNIT', 'ST', 'STATE',
       'ST_ABBR', 'STCNTY', 'COUNTY', 'FIPS',
       ...
       'EP_ASIAN', 'MP_ASIAN', 'EP_AIAN', 'MP_AIAN', 'EP_NHPI', 'MP_NHPI',
       'EP_TWOMORE', 'MP_TWOMORE', 'EP_OTHERRACE', 'MP_OTHERRACE'],
      dtype='object', length=160)

In [ ]:
cdc_eagle_fips = EAGLE_outages2020.merge(cdc_socialVulnerability2020, left_on='fips_code', right_on='FIPS', how='inner')